In [1]:
import glob
import os
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime

from corp_bond_modules import get_corporate_bond_holdings
from corp_bond_modules import weighted_quantile

In [2]:
url = "https://www.vanguard.co.uk/professional/product/etf/bond/9594/usd-corporate-bond-ucits-etf-usd-accumulating"
path = "./VDPA"

get_corporate_bond_holdings(url,os.path.abspath(path))

Initializing Chrome webdriver...
Navigating to https://www.vanguard.co.uk/professional/product/etf/bond/9594/usd-corporate-bond-ucits-etf-usd-accumulating...
Extracting holdings data...
File downloaded


VDPA

In [3]:
path = "./VDPA"
path += "/*"

target_cols = ['Ticker', 'Holding', 'Market_Val_Percent', 'Market_Val_USD', 
        'Face_Val_USD', 'Coupon_Percent', 'Maturity_Date']

expected_cols = ['Ticker','Holding name','% of market value',
                 'Market value','Face amount','Coupon/Yield','Maturity date']

cols = dict(zip(expected_cols,target_cols))

dateparse = lambda x: datetime.strptime(x, '%d %b %Y')

df = pd.read_excel(max(glob.glob(path), key=os.path.getctime),
    skiprows=6)[:-2].rename(columns=cols)

df['Market_Val_Percent'] = df['Market_Val_Percent'].apply(lambda x: float(x.replace('%', '')))
df['Market_Val_USD'] = df['Market_Val_USD'].apply(lambda x: float(str(x).replace('US$', '').replace(',', '')))
df['Face_Val_USD']=df['Face_Val_USD'].apply(lambda x: float(str(x).replace(',', '')))
df['Coupon_Percent']=df['Coupon_Percent'].apply(lambda x: float(str(x).replace('%', '')))
df['Maturity_Date']=df['Maturity_Date'].apply(lambda x: pd.to_datetime(x,format='%d %b %Y'))
df['Maturity_Years_FromNow']=df['Maturity_Date'].apply(lambda x:(x-datetime.today()).days/365.25 if x.year > datetime.now().year else 0)
df['AnnExpectCoupon_NomUSD'] = df['Face_Val_USD']*df['Coupon_Percent']/100

year_bins = (
    np.pad(weighted_quantile(df['Maturity_Years_FromNow'],quantiles=[0.25,0.5,0.75],sample_weight=df['Market_Val_Percent']), 
           (1, 1), mode='constant', 
           constant_values=(0, int(np.ceil(df['Maturity_Years_FromNow'].max())))
           )
)

df['Year_Category']=pd.cut(df['Maturity_Years_FromNow'],bins=year_bins)

In [4]:
df

,Ticker,Holding,Market_Val_Percent,Market_Val_USD,Face_Val_USD,Coupon_Percent,Maturity_Date,Maturity_Years_FromNow,AnnExpectCoupon_NomUSD,Year_Category
0,T,United States Treasury Note/Bond,0.2185,7412074.67,7417000.0,4.13,2026-10-31,1.952088,306322.1,"(0.0, 3.369]"
1,ABIBB,Anheuser-Busch Cos LLC / Anheuser-Busch InBev ...,0.0988,3352727.09,3595000.0,4.90,2046-02-01,21.207392,176155.0,"(15.417, 149.0]"
2,COP,ConocoPhillips Co,0.0755,2559860.83,2337000.0,6.95,2029-04-15,4.407940,162421.5,"(3.369, 6.407]"
3,CVS,CVS Health Corp,0.0740,2509296.73,2905000.0,5.05,2048-03-25,23.351129,146702.5,"(15.417, 149.0]"
4,TMUS,T-Mobile USA Inc,0.0728,2469807.42,2600000.0,3.88,2030-04-15,5.407255,100880.0,"(3.369, 6.407]"
...,...,...,...,...,...,...,...,...,...,...
9786,TOYOTA,Toyota Motor Credit Corp,0.0000,0.00,0.0,4.05,2025-10-24,0.933607,0.0,"(0.0, 3.369]"
9787,HKLSP,Hongkong Land Finance Cayman Islands Co Ltd/The,0.0000,0.00,0.0,4.50,2025-10-07,0.887064,0.0,"(0.0, 3.369]"
9788,STOAU,Santos Finance Ltd,0.0000,0.00,0.0,5.25,2029-03-13,4.317591,0.0,"(3.369, 6.407]"
9789,HONHAI,Foxconn Far East Ltd,0.0000,0.00,0.0,1.63,2025-10-28,0.944559,0.0,"(0.0, 3.369]"


In [5]:
fig = px.histogram(df,x='Coupon_Percent',y='Market_Val_Percent',color='Year_Category',nbins=100,barmode='overlay')
fig.show()

In [6]:
# Adjusted coupon based on current market value (not face value), reflecting real yield??
# NPV of coupons, compare price?

In [7]:
fig = px.ecdf(df,x='Maturity_Years_FromNow',y='Market_Val_Percent',marginal='histogram')
fig.show()

In [8]:
# Mean

df2 = df[df['Coupon_Percent']>0]
sum((df2['Market_Val_Percent']/sum(df2['Market_Val_Percent']))*df2['Coupon_Percent'])

4.326343408916992

In [9]:
# Median

weighted_quantile(df['Coupon_Percent'],quantiles=[0.5],sample_weight=df['Market_Val_Percent'])

array([4.41])